* ToDo
    + Evaluationの実装 : ongoing (-1/20)
    + TrainManagerの改良 : ongoing (-1/31)
    + Metricsの定義 : not start (1/24-1/25)
    + Eventの追加 : not start (1/25-1/31)
        - EarlyStopping
        - ModelSave
    + RL : not start (2/1-2/7)
    + LRScheduller : not start (1/26-1/31)
    + GPU : not start ()
    + DataParalell : not start ()
    + ModelParalell : not start ()

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../')

In [3]:
import argparse
parser = argparse.ArgumentParser()
parser.add_argument("--mnist-path")
args = parser.parse_args(args=["--mnist-path", "/home/noriaki/MachineLearning/Datasets/"])


import numpy as np
np.random.seed(10)
from develop.flash.debug import get_data_loaders
mnist_path = args.mnist_path
train_loader, val_loader = get_data_loaders(train_batch_size=16,
                                            val_batch_size=16, mnist_path=mnist_path,
                                            train_dataset_size=np.random.randint(0,60000,60),
                                            val_dataset_size=np.random.randint(0,10000,23), download=True)

from develop.flash.dataloader import get_sampled_loader
eval_train_loader = get_sampled_loader(train_loader, num_samples=35)


from develop.flash.debug import TestNet
model = TestNet()

from develop.flash.optimizers import get_optimzier

optimizer_info = {
    "name" : "SGD",
    "args" : {
        "lr" : 0.01,
        "momentum" : 0.5    
    },
}

optimizer = get_optimzier(optimizer_info, model)

5 5


In [4]:
from develop.flash.utils import wrap_metrics, get_y_values
import torch.nn as nn

loss_ = wrap_metrics(nn.NLLLoss(), get_y_values)

In [5]:
from flash.core.data_utils import DataGateway, DataWarehouse, TrainDataServer
from flash.core.process import ModelFlowProcess, Process, ModelFlowManager, EventManager

In [6]:
# new config ???
{
    "data-gateway" : {
        "num_batch_division" : 1,
        "max_batch"
    }
}

SyntaxError: invalid syntax (<ipython-input-6-813f4a2febdc>, line 6)

In [6]:
config = {
    "train" : {
        "batch_size" : 32
    },
    "data" : {
        "input_transform" : lambda x:{'x':x[0], 'y':x[1]}
    },
    "others" : {
        "grad_accumulation_steps" : 1,
        "eval_batch_size" : 32
    }
}

In [7]:
# temporal
"""
    configのデフォルト値はどうする？
"""

def create_default_data_server(config):
    update_data_gateway_args = dict(
        default_input_size = config['train']['batch_size'],
        num_batch_division = config['others'].get('grad_accumulation_steps', 1),
        input_transform = config['data'].get('input_transform', None)
    )
    eval_data_gateway_args = dict(
        default_input_size = config['others']['eval_batch_size'],
        input_transform = config['data'].get('input_transform', None)
    )
    
    update_data_gateway = DataGateway(**update_data_gateway_args)
    eval_data_gateway = DataGateway(**eval_data_gateway_args)
    data_warehouse = DataWarehouse()
    update_data_server = TrainDataServer(data_gateway=update_data_gateway, data_warehouse=data_warehouse)
    eval_data_server = TrainDataServer(data_gateway=eval_data_gateway, data_warehouse=data_warehouse)
    return { 'update' : update_data_server, 'evaluate' : eval_data_server }

dg = DataGateway(default_input_size=10, max_batch_size=10, num_batch_division=2, input_transform=lambda x:{'x':x[0], 'y':x[1]})

dw = DataWarehouse()

ds = TrainDataServer(data_gateway=dg, data_warehouse=dw, data_loader=eval_train_loader)

from collections import OrderedDict

In [ ]:
from flash.core.process import ProcessManager
class EvalModelFlowManager(ProcessManager):
    def __init__(self, data_server, event_manager):
        self.data_server = data_server
        #self.event_manager = event_manager
        self.process_info = OrderedDict()
        self.state = {}# class State in the future
        
        self.state["global/epoch"] = 1
        self.state["global/iteration"] = 1
    
    def register_process(self, process, process_name=None, process_config={}):
        # type name : str
        assert hasattr(process,'run')

        if process_name is None:
            process_name = f"process-{len(self.process_info)+1}"
        
        if hasattr(process, 'name'):
            assert process_name == getattr(process, 'name', process_name)
        else:
            setattr(process, 'name', process_name)
        assert process_name not in self.process_info.keys()
        
        # (process, process_config)
        self.process_info.update({process_name : (process, process_config)})
        
    def register_multiple_processes(self, process_info):
        raise NotImplementedError
        # assert hasattr(process,run)
        #assert process_name not in self.process_info.keys()
        
        # (process, process_config)
        # may change into list from dict because Process class has its name ...
        self.process_info.update(process_info)
        
    def __iter__(self):
        return enumerate(self.process_info.values(), 1)
        
    def _reset_state(self):
        self.state["run/epoch"] = 1
        self.state["run/iteration"] = 1
    
    def run(self, state=None, process_config=dict(num_epochs=1)):
        num_epochs = process_config['num_epochs']
        data_loader_name = process_config.get('data_loader_name', None)## eval

        # ToDo: Consider the training restart case
        self._reset_state()
        state = self.state
        
        self._set_data_server()
        data_server = self.data_server
        if data_loader_name is not None:### eval
            data_server.set_data_loader(data_loader_name)### eval
        #event_manager = self.event_manager
        #event_manager._set_state(state)
        
        #event_manager.fire_events("run-start")
        for epoch in range(1, num_epochs+1):
            #event_manager.fire_events("epoch-start")
            
            for iteration in data_server:
                #event_manager.fire_events("iter-start")
                #self.state.output = self._process_function(self, self.state.batch)

                for N, (process, process_config_) in self:
                    process_config_ = {**process_config, **process_config_}
                    process.run(state=state, process_config=process_config_)
                #event_manager.fire_events("iter-end")
                #results = data_server.get_results()
                self.state["run/iteration"] = iteration
                self.state["global/iteration"] += 1
                
            #event_manager.fire_events("epoch-end")
            self.state["run/epoch"] = epoch
            self.state["global/epoch"] += 1
            # evnet on exceptions
        #event_manager.fire_events("run-end")


In [11]:
# new version : TrainManager

class TrainManager():
    def __init__(self):
        pass

    def set_config(self, config: dict):
        self.config = config
        #self.check()
        self.config['objects'] = {
            'update_process_list' : [],
            'evaluate_process_list' : [],
        }#temporal

    def setup(self):
        config = self.config
        data = config['objects']['data']
        
        data_server_set = create_default_data_server(config)
        event_manager = EventManager(data_server=None)
        update_process_manager = ModelFlowManager(data_server=data_server_set['update'], event_manager=event_manager)
        evaluate_process_manager = ModelFlowManager(data_server=data_server_set['evaluate'], event_manager=None)
        
        self.data_server_set = data_server_set
        self.event_manager = event_manager
        self.update_process_manager = update_process_manager
        self.evaluate_process_manager = evaluate_process_manager
        
        for process in self.update_process_list:
            update_process_manager.register_process(process)
        for process in self.evaluate_process_list:
            evaluate_process_manager.register_process(process)
        event_manager.register_event("epoch-end", evaluate_process_manager, {'num_epochs' : 1, 'data_loader_name' : 'eval_train', 'output_save_info' : {'Save_all':True}})
        event_manager.register_event("epoch-end", evaluate_process_manager, {'num_epochs' : 1, 'data_loader_name' : 'val', 'output_save_info' : {'Save_all':True}})
        
        data_server_set['update'].register_data_loader({'train':data['train_loader']})
        data_server_set['evaluate'].register_data_loader({'eval_train':data['eval_train_loader'], 'val':data['val_loader']})

    def add_process(self, process, mode='update'):
        assert mode in ['update', 'evaluate']
        #self.process_manager.register_process(process)
        
        config = self.config
        if isinstance(process, list):
            raise NotImplementedError
        elif isinstance(process, dict):
            raise NotImplementedError
        else:
            process_list = [process]
            # diagnosis_process(process)
        
        if mode == 'update':
            update_process_list = config['objects']['update_process_list']
            update_process_list.extend(process_list)
        elif mode == 'evaluate':
            evaluate_process_list = config['objects']['evaluate_process_list']
            evaluate_process_list.extend(process_list)
    
    @property
    def update_process_list(self):
        return self.config['objects']['update_process_list']

    @property
    def evaluate_process_list(self):
        return self.config['objects']['evaluate_process_list']

    def set_dataloader(self, train_loader, val_loader, eval_train_loader=None):
        config = self.config
        data = {
            'train_loader' : train_loader,
            'val_loader' : val_loader
        }
        if eval_train_loader is not None:
            data.update({'eval_train_loader' : eval_train_loader})
        config["objects"].update({ 'data' : data })

In [20]:
from flash.core.data_utils import TrainDataServer
from flash.core.process import ModelFlowProcess, ModelFlowManager

In [21]:
update_process = ModelFlowProcess(model, loss_fn=loss_, optimizer=optimizer, mode='train', process_name='up')
eval_process = ModelFlowProcess(model, mode='eval', process_name='ev')

In [22]:
tm = TrainManager()
tm.set_config(config)
tm.set_dataloader(train_loader=train_loader, val_loader=val_loader, eval_train_loader=eval_train_loader)

In [23]:
tm.add_process(update_process, mode='update')
tm.add_process(eval_process, mode='evaluate')

In [24]:
tm.setup()

In [25]:
ds = tm.data_server_set['update']
ds.set_data_loader(key='train')

ds.results, tm.update_process_manager.state

({}, {'global/epoch': 1, 'global/iteration': 1})

In [26]:
tm.update_process_manager.run()

1 {'num_epochs': 1} {}
up train {'num_epochs': 1}
1 {'num_epochs': 1} {}
up train {'num_epochs': 1}
1 {'num_epochs': 1} {}
up train {'num_epochs': 1}
1 {'num_epochs': 1} {}
up train {'num_epochs': 1}
1 {'num_epochs': 1, 'data_loader_name': 'eval_train', 'output_save_info': {'Save_all': True}} {}
ev eval_train {'num_epochs': 1, 'data_loader_name': 'eval_train', 'output_save_info': {'Save_all': True}}
1 {'num_epochs': 1, 'data_loader_name': 'eval_train', 'output_save_info': {'Save_all': True}} {}
ev eval_train {'num_epochs': 1, 'data_loader_name': 'eval_train', 'output_save_info': {'Save_all': True}}
1 {'num_epochs': 1, 'data_loader_name': 'eval_train', 'output_save_info': {'Save_all': True}} {}
ev eval_train {'num_epochs': 1, 'data_loader_name': 'eval_train', 'output_save_info': {'Save_all': True}}
1 {'num_epochs': 1, 'data_loader_name': 'val', 'output_save_info': {'Save_all': True}} {}
ev val {'num_epochs': 1, 'data_loader_name': 'val', 'output_save_info': {'Save_all': True}}
1 {'num_e

In [27]:
ds.data_warehouse.data_stack

defaultdict(<function flash.core.data_utils.DataWarehouse._reset.<locals>.<lambda>()>,
            {'currnt/outputs': [{'y': tensor([[-2.3958, -2.5058, -2.4579, -2.0148, -2.3350, -2.2528, -2.3071, -2.3932,
                        -2.1541, -2.3076],
                       [-2.2914, -2.4203, -2.4016, -2.1853, -2.3184, -2.2392, -2.3318, -2.3759,
                        -2.2049, -2.2860],
                       [-2.3856, -2.4869, -2.4721, -1.9957, -2.3032, -2.2534, -2.3143, -2.3649,
                        -2.1854, -2.3616],
                       [-2.3482, -2.4150, -2.4081, -2.0906, -2.3258, -2.3125, -2.2777, -2.3726,
                        -2.2008, -2.3198],
                       [-2.3275, -2.5099, -2.5066, -2.0196, -2.3333, -2.2271, -2.3286, -2.3570,
                        -2.1708, -2.3443],
                       [-2.3580, -2.4298, -2.4748, -2.0837, -2.2764, -2.2359, -2.3355, -2.3533,
                        -2.1770, -2.3650],
                       [-2.3249, -2.4509, -2.5059, -2.01

In [28]:
from flash.utils import _concat_results

In [47]:
tmp = ds.data_warehouse.data_stack['currnt/outputs']

In [49]:
tmp[0]['y'].shape

torch.Size([16, 10])

In [35]:
#y = _concat_results(ds.data_warehouse.data_stack['current/outputs'])
y = _concat_results(ds.data_warehouse.data_stack['currnt/outputs'])

In [37]:
y = y['y']

In [38]:
y

tensor([[-2.3958, -2.5058, -2.4579, -2.0148, -2.3350, -2.2528, -2.3071, -2.3932,
         -2.1541, -2.3076],
        [-2.2914, -2.4203, -2.4016, -2.1853, -2.3184, -2.2392, -2.3318, -2.3759,
         -2.2049, -2.2860],
        [-2.3856, -2.4869, -2.4721, -1.9957, -2.3032, -2.2534, -2.3143, -2.3649,
         -2.1854, -2.3616],
        [-2.3482, -2.4150, -2.4081, -2.0906, -2.3258, -2.3125, -2.2777, -2.3726,
         -2.2008, -2.3198],
        [-2.3275, -2.5099, -2.5066, -2.0196, -2.3333, -2.2271, -2.3286, -2.3570,
         -2.1708, -2.3443],
        [-2.3580, -2.4298, -2.4748, -2.0837, -2.2764, -2.2359, -2.3355, -2.3533,
         -2.1770, -2.3650],
        [-2.3249, -2.4509, -2.5059, -2.0117, -2.3014, -2.2562, -2.3456, -2.4235,
         -2.2201, -2.2754],
        [-2.4154, -2.4763, -2.4381, -2.1225, -2.3087, -2.2559, -2.2550, -2.2869,
         -2.1698, -2.3567],
        [-2.3611, -2.4791, -2.4545, -2.0923, -2.3321, -2.2939, -2.2856, -2.3680,
         -2.1486, -2.2772],
        [-2.3379, -

In [40]:
softmax = nn.Softmax(dim=1)
nllloss = nn.NLLLoss()

In [43]:
p = softmax(y)

In [44]:
p

tensor([[0.0911, 0.0816, 0.0856, 0.1334, 0.0968, 0.1051, 0.0995, 0.0913, 0.1160,
         0.0995],
        [0.1011, 0.0889, 0.0906, 0.1124, 0.0984, 0.1065, 0.0971, 0.0929, 0.1103,
         0.1017],
        [0.0920, 0.0832, 0.0844, 0.1359, 0.0999, 0.1050, 0.0988, 0.0940, 0.1124,
         0.0943],
        [0.0955, 0.0894, 0.0900, 0.1236, 0.0977, 0.0990, 0.1025, 0.0932, 0.1107,
         0.0983],
        [0.0975, 0.0813, 0.0815, 0.1327, 0.0970, 0.1078, 0.0974, 0.0947, 0.1141,
         0.0959],
        [0.0946, 0.0881, 0.0842, 0.1245, 0.1027, 0.1069, 0.0968, 0.0951, 0.1134,
         0.0939],
        [0.0978, 0.0862, 0.0816, 0.1338, 0.1001, 0.1047, 0.0958, 0.0886, 0.1086,
         0.1028],
        [0.0893, 0.0841, 0.0873, 0.1197, 0.0994, 0.1048, 0.1049, 0.1016, 0.1142,
         0.0947],
        [0.0943, 0.0838, 0.0859, 0.1234, 0.0971, 0.1009, 0.1017, 0.0937, 0.1166,
         0.1026],
        [0.0965, 0.0837, 0.0876, 0.1252, 0.0960, 0.1037, 0.0999, 0.0966, 0.1149,
         0.0960],
        [0

In [ ]:
torch.argmax()

In [268]:
ds1.current_buffer

defaultdict(<function flash.core.data_utils.TrainDataServer.__init__.<locals>.<lambda>()>,
            {'outputs': {'up': {'y': tensor([[-3.0331, -1.2355, -2.3531, -2.3925, -2.4973, -3.2562, -2.7663, -2.3941,
                        -2.0924, -2.5733],
                       [-3.4033, -2.6814, -2.1397, -1.6921, -2.9154, -1.9642, -2.8226, -1.9775,
                        -2.5565, -2.0670],
                       [-2.9693, -1.6624, -2.9843, -2.0127, -2.8591, -2.9935, -2.0942, -2.7243,
                        -1.6810, -2.3818],
                       [-2.4177, -2.9134, -2.3364, -1.6145, -2.5476, -2.5784, -2.1098, -2.2998,
                        -2.3147, -2.4487],
                       [-2.5840, -2.0995, -2.4297, -2.2582, -2.2021, -2.6670, -2.1887, -2.4873,
                        -2.1505, -2.1392],
                       [-2.6444, -2.4692, -2.6925, -1.8929, -2.8744, -2.1172, -1.9657, -2.4038,
                        -2.2271, -2.1995],
                       [-3.1309, -1.9826, -1.7994, -1

In [151]:
class Temp():
    def __init__(self, l):
        self.list = l
        
    def __iter__(self):
        print("iter")
        self._l = iter(self.list)
        return self
    
    def __next__(self):
        return next(self._l)

In [152]:
tmp = Temp([1,2,3,4])

In [153]:
for i in tmp:
    print(i)

iter
1
2
3
4


In [154]:
for i in tmp:
    print(i)

iter
1
2
3
4


In [63]:
ds2 = tm.data_server_set['evaluate']
ds2.results

{'outputs/ev/y': tensor([[-2.4509, -2.3101, -2.3044, -1.8983, -2.6456, -2.3190, -2.4541, -2.1979,
          -2.4048, -2.2229],
         [-2.4792, -2.2891, -2.2637, -1.9546, -2.6114, -2.2988, -2.4784, -2.1929,
          -2.3427, -2.2645],
         [-2.4481, -2.2974, -2.3388, -1.9652, -2.5741, -2.3655, -2.4591, -2.1345,
          -2.3696, -2.2164],
         [-2.5014, -2.3376, -2.3617, -1.8615, -2.5707, -2.3960, -2.4191, -2.2114,
          -2.3710, -2.1870],
         [-2.4859, -2.2364, -2.2402, -2.0082, -2.3785, -2.3877, -2.3664, -2.2785,
          -2.4692, -2.2652],
         [-2.3668, -2.3091, -2.2742, -1.9547, -2.4272, -2.4129, -2.3876, -2.3449,
          -2.4199, -2.2265],
         [-2.4660, -2.2338, -2.2527, -1.9562, -2.4804, -2.4166, -2.3631, -2.2506,
          -2.3944, -2.3246]])}

In [65]:
ds2.current_data_loader_name

'val'

In [25]:
ds.current_buffer

defaultdict(<function flash.core.data_utils.TrainDataServer.__init__.<locals>.<lambda>()>,
            {'outputs': {'model': {'y': tensor([[-1.9741, -2.2178, -2.4961, -2.2982, -2.4555, -2.2134, -2.4460, -2.5294,
                        -2.2897, -2.2383],
                       [-2.4829, -2.1912, -2.3693, -2.0773, -2.3470, -2.6201, -2.3363, -2.2773,
                        -2.1844, -2.2476],
                       [-2.2419, -2.2760, -2.3020, -2.3078, -2.1298, -2.2984, -2.3899, -2.4092,
                        -2.4792, -2.2360],
                       [-2.3620, -2.3347, -2.2606, -1.9512, -2.4382, -2.2875, -2.2999, -2.2435,
                        -2.7872, -2.2465],
                       [-2.2266, -2.0384, -2.2657, -2.2339, -2.4156, -2.4095, -2.2907, -2.4704,
                        -2.4852, -2.2752],
                       [-2.2922, -2.3219, -2.4574, -2.2366, -2.4117, -2.5749, -2.1518, -1.9447,
                        -2.4090, -2.3713],
                       [-2.1283, -2.1718, -2.3789,

In [26]:
ds2.current_buffer

defaultdict(<function flash.core.data_utils.TrainDataServer.__init__.<locals>.<lambda>()>,
            {'outputs': {'model': {'y': tensor([[-2.2850, -2.2918, -2.2173, -2.1412, -2.5096, -2.4006, -2.4183, -2.2720,
                        -2.3378, -2.2070],
                       [-2.3001, -2.2240, -2.2181, -2.2016, -2.4948, -2.3798, -2.4446, -2.2887,
                        -2.2776, -2.2408],
                       [-2.2259, -2.2514, -2.2707, -2.2234, -2.4646, -2.4029, -2.3824, -2.2798,
                        -2.3384, -2.2194],
                       [-2.2863, -2.2375, -2.2621, -2.1776, -2.4440, -2.4381, -2.3589, -2.3477,
                        -2.2791, -2.2298],
                       [-2.3760, -2.1657, -2.2414, -2.1750, -2.3041, -2.3731, -2.3718, -2.3781,
                        -2.3954, -2.2802],
                       [-2.2139, -2.2473, -2.2627, -2.1829, -2.3155, -2.4565, -2.3906, -2.3607,
                        -2.3766, -2.2537],
                       [-2.3297, -2.1529, -2.2514,

In [24]:
ds2.data_warehouse.data_tree

In [27]:
tm.evaluate_process_manager.process_info

OrderedDict([('process-1',
              (<flash.core.process.ModelFlowProcess at 0x7f7ee3001978>, {}))])

In [100]:
tmp = tm.update_process_manager.process_info

In [106]:
for i in ds:
    print(i)

TypeError: 'NoneType' object is not iterable

In [104]:
tmp['process-1'][0].run()

AttributeError: 'TrainDataServer' object has no attribute 'inputs'

In [62]:
ds = tm.data_server_set['update']

ds.results

{}

In [28]:
ds = tm.data_server_set['update']

ds.results

{}

In [78]:
hasattr(update_process, 'name')

False

In [77]:
update_process.name

AttributeError: 'ModelFlowProcess' object has no attribute 'name'

In [29]:
%autoreload 2

In [ ]:
class Temp():
    def add_process(self):
        pass
    
    def reset_update_info(self):
        config = self.config
        config['trainer']['update_process_list'] = []

    def reset_evaluate_info(self):
        config = self.config
        config['trainer']['evaluate_process_list'] = []

    def add_update_info(self, **update_info):
        assert 'model' in update_info
        #assert 'loss_fn' in update_info
        assert all(map(lambda x:isinstance(x, str) or isinstance(x, int) or isinstance(x, float), update_info.values()))
        config = self.config
        update_process_list = config['trainer']['update_process_list']
        update_process_list.append(update_info)
        
    def add_evaluate_info(self, **evaluate_info):
        assert 'model' in evaluate_info
        assert all(map(lambda x:isinstance(x, str), evaluate_info.values()))
        config = self.config
        evaluate_process_list = config['trainer']['evaluate_process_list']
        evaluate_process_list.append(evaluate_info)


    def set_objects(self):
        import copy
        config = self.config
        trainer = config['trainer']
        objects = config['objects']
        key2obj = {
            'model' : objects['models'],
            'loss_fn' : objects['loss_fns'],
            'optimizer' : objects['optimizers'],
            'skip_condition' : objects['skip_condition'],
            'break_condition' : objects['break_condition'],
            'pre_operator' : objects['pre_operator'],
            'post_operator' : objects['post_operator'],
        }

        update_process_list = []
        for update_info in trainer['update_process_list']:
            update_info_ = copy.copy(update_info)
            for key, obj in key2obj.items():
                if key in update_info:
                    assert update_info[key] in obj.keys(), f"'{key}' key '{update_info[key]}' is not registered"
                    update_info_[key] = obj[update_info[key]]
            update_process_list.append(update_info_)
        objects.update({'update_process_list' : update_process_list})

        key2obj = {
            'model' : objects['models'],
            'skip_condition' : objects['skip_condition'],
            'pre_operator' : objects['pre_operator'],
            'post_operator' : objects['post_operator'],
        }

        evaluate_process_list = []
        for evaluate_info in trainer['evaluate_process_list']:
            evaluate_info_ = copy.copy(evaluate_info)
            for key, obj in key2obj.items():
                if key in evaluate_info:
                    assert evaluate_info[key] in obj.keys(), f"'{key}' key '{evaluate_info[key]}' is not registered"
                    evaluate_info_[key] = obj[evaluate_info[key]]
            evaluate_process_list.append(evaluate_info_)
        objects.update({'evaluate_process_list' : evaluate_process_list})


    def setup_engines(self, trainer_args={}, evaluator_args={}):
        #from flash.engine import create_trainer, create_evaluator
        from develop.flash.engine import create_trainer, create_evaluator
        config = self.config
        batch_size = config['train']['batch_size']###
        #eval_batch_size = config['others']['eval_batch_size']###
        objects = config['objects']
        device = config['device']['name']
        grad_accumulation_steps = config['others'].get('grad_accumulation_steps', 1)
        metrics = objects['metrics']
        #train_loader = objects['data']['train_loader']
        update_process_list = objects['update_process_list']
        #update_info_list = objects['update_info_list']
        evaluate_process_list = objects['evaluate_process_list']
        #evaluate_info_list = objects['evaluate_info_list']

        trainer = create_trainer(process_list=update_process_list, batch_size=batch_size, **trainer_args)
        train_evaluator = create_evaluator(process_list=evaluate_process_list, metrics=metrics, **evaluator_args)
        evaluator_args.update({ 'data_server' : train_evaluator.data_server })# temporal
        val_evaluator = create_evaluator(process_list=evaluate_process_list, metrics=metrics, **evaluator_args)

        objects['engine'] = {
            'trainer' : trainer,
            'train_evaluator' : train_evaluator,
            'val_evaluator' : val_evaluator
        }

em = EventManager(data_server=ds)

pm = ProcessManager(data_server=ds, event_manager=em)
pm.register_process(process)

In [19]:
model.conv1.weight[0]

tensor([[[-0.2786, -0.1996, -0.0905],
         [ 0.1691,  0.1847, -0.1437],
         [-0.3222, -0.2673,  0.1372]]], grad_fn=<SelectBackward>)

In [20]:
pm.run(1)

In [21]:
model.conv1.weight[0]

tensor([[[-0.2788, -0.1997, -0.0905],
         [ 0.1692,  0.1847, -0.1435],
         [-0.3222, -0.2678,  0.1369]]], grad_fn=<SelectBackward>)

In [45]:
pm.state

{'global/epoch': 3,
 'global/iteration': 41,
 'run/epoch': 0,
 'run/iteration': 20}

In [ ]:
# old version

import os
from collections import defaultdict

import torch
from ignite.metrics import Accuracy

from flash.utils import wrap_metrics, get_y_values
from flash.metrics import Loss
from flash.files import prepare_target_dir



"""
    ToDo
    * assign a device to each model model
    * add the following properties
        * __repr__
        * __str__
        * __dict__
        * __getter__
        * __setter__
        * quick access to the field in config
            + manager.config.objects.engine.trainer, for example
"""
class TrainManager():
    def __init__(self):
        pass

    """experimental"""
    def load_config(self, config_path):
        ext = os.path.splitext(config_path)[1]
        if ext == ".json":
            import json
            with open(config_path, "r") as f:
                config = json.load(f)
        elif ext == ".toml":
            import toml
            with open(config_path, "r") as f:
                config = toml.load(f)
        elif ext == ".yaml" or ext == ".yml":
            import yaml
            with open(config_path, "r") as f:
                config = yaml.safe_load(f)
        else:
            raise NotImplementedError
        self.set_config(config)

    """experimental"""
    def set_seed(self, seed_dict={}):
        if seed_dict is not None:
            import random
            import numpy as np
            import torch
            radom.seed(seed_dict.get("random", 0))
            np.radom.seed(seed_dict.get("numpy", 0))
            torch.manual_seed(seed_dict.get("torch", 0))

    def set_config(self, config: dict):
        self.config = config
        self.check()

    def add_model_from_generator(self, model_name:str, model_generator, model_args={}, device=None):
        assert isinstance(model_name ,str)
        config = self.config
        model = model_generator(**model_args)
        device = config['device']['name'] if device is None else device
        if device == 'cuda':
            model.cuda(config['device']['gpu'])
        config['objects']['models'].update({model_name : model})
        
        """
        if config['device']['num_gpu'] > 0:
            device = 'cuda' if torch.cuda.is_available() else 'cpu'
            #if device == 'cuda':
            #    model.cuda(config['device']['gpu'])
            config['device']['name'] = device
        else:
            config['device']['name'] = 'cpu'
        
        config['objects']['model'] = model
        """

    def add_model(self, model_name:str, model, device=None):
        assert isinstance(model_name ,str)
        config = self.config
        # temporal
        if device is None:
            device_cfg = self.config['device']
            num_gpu = device_cfg.get('num_gpu', 1)
            if num_gpu > 0:
                device = device_cfg.get('gpu', 0)
            else:
                device = -1
        config['objects']['models'].update({model_name : model.cuda(device) if torch.cuda.is_available() or device >= 0 else model})

    def add_object(self, obj_key:str, obj_name:str, obj):
        assert isinstance(obj_key ,str)
        assert isinstance(obj_name ,str)
        config = self.config
        config['objects'][obj_key].update({obj_name : obj})

    def add_optimizer(self, optimizer_name:str, optimizer):
        assert isinstance(optimizer_name ,str)
        config = self.config
        config['objects']['optimizers'].update({optimizer_name : optimizer})

    # keep
    def add_loss_fn_old(self, loss_fn_name, loss_fn, output_transform=get_y_values):
        assert isinstance(loss_fn_name ,str)
        config = self.config
        loss_fn_ = wrap_metrics(loss_fn, output_transform)
        config['objects']['loss_fns'].update({loss_fn_name : loss_fn_})

    def add_loss_fn(self, loss_fn_name, loss_fn):
        assert isinstance(loss_fn_name ,str)
        config = self.config
        config['objects']['loss_fns'].update({loss_fn_name : loss_fn})

    def add_skip_condition(self, skip_condition_name, skip_condition):
        assert isinstance(skip_condition_name ,str)
        config = self.config
        config['objects']['skip_condition'].update({skip_condition_name : skip_condition})

    def add_break_condition(self, break_condition_name, break_condition):
        assert isinstance(break_condition_name ,str)
        config = self.config
        config['objects']['break_condition'].update({break_condition_name : break_condition})

    def add_pre_operator(self, pre_operator_name, pre_operator):
        assert isinstance(pre_operator_name ,str)
        config = self.config
        config['objects']['pre_operator'].update({pre_operator_name : pre_operator})

    def add_post_operator(self, post_operator_name, post_operator):
        assert isinstance(post_operator_name ,str)
        config = self.config
        config['objects']['post_operator'].update({post_operator_name : post_operator})

    def reset_update_info(self):
        config = self.config
        config['trainer']['update_process_list'] = []

    def reset_evaluate_info(self):
        config = self.config
        config['trainer']['evaluate_process_list'] = []

    def add_update_info(self, **update_info):
        assert 'model' in update_info
        #assert 'loss_fn' in update_info
        assert all(map(lambda x:isinstance(x, str) or isinstance(x, int) or isinstance(x, float), update_info.values()))
        config = self.config
        update_process_list = config['trainer']['update_process_list']
        update_process_list.append(update_info)
        
    def add_evaluate_info(self, **evaluate_info):
        assert 'model' in evaluate_info
        assert all(map(lambda x:isinstance(x, str), evaluate_info.values()))
        config = self.config
        evaluate_process_list = config['trainer']['evaluate_process_list']
        evaluate_process_list.append(evaluate_info)

    def add_process(self, process, mode='update'):
        if isinstance(process, list):
            raise NotImplementedError
        elif isinstance(process, dict):
            raise NotImplementedError
        else:
            process_list = [process]
            # diagnosis_process(process)
        
        config = self.config
        if mode == 'update':
            update_process_list = config['objects']['update_process_list']
            update_process_list.extend(process_list)
        elif mode == 'evaluate':
            evaluate_process_list = config['objects']['evaluate_process_list']
            evaluate_process_list.extend(process_list)

    def set_dataloader(self, train_loader, val_loader, eval_train_loader=None):
        config = self.config
        data = {
            'train_loader' : train_loader,
            'val_loader' : val_loader
        }
        if eval_train_loader is not None:
            data.update({'eval_train_loader' : eval_train_loader})
        config["objects"].update({ 'data' : data })


    # temporal ?
    def setup_metrics(self, target_metrics, loss_fn=None, output_transform=get_y_values, target_loss_fn_names=None):
        """
            ToDo
                * Add other metrics ...
                * Refactoring and make/unify the logic clear
        """
        from flash.metrics import get_precision, get_recall, get_F1score
        config = self.config
        #loss_fns = self.config['objects']['loss_fns']

        metrics = {}
        if 'loss' in target_metrics:
            if target_loss_fn_names is None:
                #if len(loss_fns) == 1:
                if loss_fn is not None:
                    #loss_fn = list(loss_fns.values())[0]
                    metrics.update({ 'loss' : Loss(loss_fn) })###
                #else:
                #    target_loss_fn_names = list(loss_fns.keys())
            #if target_loss_fn_names is not None:
            #    for name, loss_fn in loss_fns.items():
            #        metrics.update({ f"{name}-loss" : Loss(loss_fn) })###
        if 'accuracy' in target_metrics:
            metrics.update({ 'accuracy' : Accuracy(output_transform=output_transform) })
        Is_average = 'precision' in target_metrics
        Is_classwise = 'precision_class' in target_metrics
        if Is_average or Is_classwise:
            metrics.update(get_precision(Is_average=Is_average, Is_classwise=Is_classwise, output_transform=output_transform))
        Is_average = 'recall' in target_metrics
        Is_classwise = 'recall_class' in target_metrics
        if Is_average or Is_classwise:
            metrics.update(get_recall(Is_average=Is_average, Is_classwise=Is_classwise, output_transform=output_transform))
        Is_average = 'F1' in target_metrics
        Is_classwise = 'F1_class' in target_metrics
        if Is_average or Is_classwise:
            metrics.update(get_F1score(Is_average=Is_average, Is_classwise=Is_classwise, output_transform=output_transform))

        print(f"set {tuple(list(metrics.keys()))}")

        config['objects']['metrics'] = metrics


    # to rename the function later
    def check(self):
        config = self.config

        config['train']['train_batch_size'] = config['train']['batch_size']###

        ### change ? (temporal)
        if config['device']['num_gpu'] > 0:
            device = 'cuda' if torch.cuda.is_available() else 'cpu'
            #if device == 'cuda':
            #    model.cuda(config['device']['gpu'])
            config['device']['name'] = device
        else:
            config['device']['name'] = 'cpu'

        if 'objects' not in config.keys():
            config['objects'] = {}
        if 'trainer' not in config.keys():
            config['trainer'] = {}
            
        for objects_key in ['models', 'loss_fns', 'optimizers', 'skip_condition', 'break_condition', 'pre_operator', 'post_operator']:
            config['objects'].update({objects_key : {}})
        config['objects'].update({'metrics_log' : defaultdict(lambda :[])})
        #config['trainer'].update({'update_process_list' : []})
        #config['trainer'].update({'evaluate_process_list' : []})
        config['objects']['update_process_list'] = []
        config['objects']['evaluate_process_list'] = []

        self._save_dir()

    def _save_dir(self):
        """
            ToDo
                * refactoring
                * allow manager to set assert_on switching
        """
        config = self.config
        save_dir = config.get('handlers', {}).get('checkpoint', {}).get('save_dir', None)
        if save_dir is not None:
            prepare_target_dir(save_dir)
        save_dir = config.get('handlers', {}).get('output', {}).get('save_dir', None)
        if save_dir is not None:
            prepare_target_dir(save_dir)
        save_dir = config.get('others', {}).get('save_dir', None)
        if save_dir is not None:
            prepare_target_dir(save_dir)

    def set_objects(self):
        import copy
        config = self.config
        trainer = config['trainer']
        objects = config['objects']
        key2obj = {
            'model' : objects['models'],
            'loss_fn' : objects['loss_fns'],
            'optimizer' : objects['optimizers'],
            'skip_condition' : objects['skip_condition'],
            'break_condition' : objects['break_condition'],
            'pre_operator' : objects['pre_operator'],
            'post_operator' : objects['post_operator'],
        }

        update_process_list = []
        for update_info in trainer['update_process_list']:
            update_info_ = copy.copy(update_info)
            for key, obj in key2obj.items():
                if key in update_info:
                    assert update_info[key] in obj.keys(), f"'{key}' key '{update_info[key]}' is not registered"
                    update_info_[key] = obj[update_info[key]]
            update_process_list.append(update_info_)
        objects.update({'update_process_list' : update_process_list})

        key2obj = {
            'model' : objects['models'],
            'skip_condition' : objects['skip_condition'],
            'pre_operator' : objects['pre_operator'],
            'post_operator' : objects['post_operator'],
        }

        evaluate_process_list = []
        for evaluate_info in trainer['evaluate_process_list']:
            evaluate_info_ = copy.copy(evaluate_info)
            for key, obj in key2obj.items():
                if key in evaluate_info:
                    assert evaluate_info[key] in obj.keys(), f"'{key}' key '{evaluate_info[key]}' is not registered"
                    evaluate_info_[key] = obj[evaluate_info[key]]
            evaluate_process_list.append(evaluate_info_)
        objects.update({'evaluate_process_list' : evaluate_process_list})


    def setup_engines(self, trainer_args={}, evaluator_args={}):
        #from flash.engine import create_trainer, create_evaluator
        from develop.flash.engine import create_trainer, create_evaluator
        config = self.config
        batch_size = config['train']['batch_size']###
        #eval_batch_size = config['others']['eval_batch_size']###
        objects = config['objects']
        device = config['device']['name']
        grad_accumulation_steps = config['others'].get('grad_accumulation_steps', 1)
        metrics = objects['metrics']
        #train_loader = objects['data']['train_loader']
        update_process_list = objects['update_process_list']
        #update_info_list = objects['update_info_list']
        evaluate_process_list = objects['evaluate_process_list']
        #evaluate_info_list = objects['evaluate_info_list']

        trainer = create_trainer(process_list=update_process_list, batch_size=batch_size, **trainer_args)
        train_evaluator = create_evaluator(process_list=evaluate_process_list, metrics=metrics, **evaluator_args)
        evaluator_args.update({ 'data_server' : train_evaluator.data_server })# temporal
        val_evaluator = create_evaluator(process_list=evaluate_process_list, metrics=metrics, **evaluator_args)

        objects['engine'] = {
            'trainer' : trainer,
            'train_evaluator' : train_evaluator,
            'val_evaluator' : val_evaluator
        }


    def create_default_events(self):
        from flash.event import create_default_events
        create_default_events(self.config)

    def run(self):
        config = self.config
        seed = self.config.get('seed', {})
        objects = config['objects']
        trainer = objects['engine']['trainer']
        train_loader = objects['data']['train_loader']
        epochs = config['train']['epochs']

        self.set_seed(seed.get('run', {}))
        trainer.run(data=train_loader, max_epochs=epochs)
        self.save_metrics()

    def save_metrics(self):
        import json
        import os
        config = self.config
        others = config['others']
        objects = config['objects']
        if 'save_dir' in others:
            metrics_log = objects['metrics_log']
            save_path = os.path.join(others['save_dir'], 'metrics_log.json')
            with open(save_path, 'w') as f:
                json.dump(metrics_log, f)


    def close(self):
        from flash.event import close_logger
        config = self.config
        others = config['others']
        objects = config['objects']
        self.save_metrics()
        
        close_logger(objects['vis_tool'], others['vis_tool'])



# pytorch dependent
class EvalProcess(Process):
    def __init__(self, model, pre_operator=None, post_operator=None,
                 skip_condition=None, process_name=None, mode='eval',
                 keep_outputs=lambda iter_, num_iter, state, name:(True, str(name)+f'{iter_}', {'retain_comp_graph':False}), metrics={}):
        # Processに移すべき？
        self.process_name = process_name
        # 必要？
        self.mode = mode
        
        self.model = model
        
        self.skip_condition = skip_condition
        self.pre_operator = pre_operator
        self.post_operator = post_operator
        
        self.keep_outputs = keep_outputs
        
        self.metrics = metrics

    def run(self, state=None, process_config={}):
        if self.skip_condition is not None:
            if self.skip_condition(self, state=state):
                return None
        
        self.model.eval()
        
        data_server = self.data_server
        params = { 'state' : state, 'iter_' : iter_, 'data_server' : data_server }
        
        if self.pre_operator is not None:
            self.pre_operator(self, **params)
            
        Keep_outputs, output_name, outputs_type_info = self.keep_outputs(iter_, self.num_iter, state, self.process_name)
        """
            original with clause as used in GPytorch
        """
        with torch.no_grad():
            for inputs, size_portion in data_server.generate_inputs():
                outputs = self(inputs, **process_config.get("forward_args", {}))
                if Keep_outputs:
                    data_server.push(outputs, transform_info=outputs_type_info, key='outputs')

        data_server.register(name_for_register=self.process_name)

        if self.post_operator is not None:
            self.post_operator(self, **params)

In [34]:
ds.results['loss/']

tensor(2.1864)

In [27]:
dw.data_tree

In [23]:
ep = EvalProcess(model=model)

In [24]:
ep.set_data_server(ds)

In [25]:
ep.run()

NameError: name 'iter_' is not defined

In [ ]:
num_train_batches = len(ds.data_loader)
grad_accumulation_steps = 1
log_interval = 2

def get_loss_from_results(results):
    return results['loss/']

import torch
def log_training_loss(self, state, process_config={}):
    """
        ### ToDo : check the following process
    """
    epoch = state["global/epoch"]
    iter_ = state["run/iteration"]
    iter_ = ((iter_ - 1) % num_train_batches) // grad_accumulation_steps + 1
    #num_iter_per_epoch = (num_train_batches - 1) // grad_accumulation_steps + 1

    if iter_ % log_interval == 0:
        results_loss = get_loss_from_results(self.data_server.results)
        #results_loss = results['loss']
        if isinstance(results_loss, torch.Tensor):
            loss_val = results_loss.item()
            print_message = "Epoch[{}] Iteration[{}/{}] Loss: {:.3f}".format(epoch, iter_, num_train_batches, loss_val)
        else:
            if isinstance(results_loss, list):
                loss_val = { f"({n})" if n != "" else "":l.item() for n,l in enumerate(results_loss, 1) }
            elif isinstance(results_loss, dict):
                loss_val = { k:l.item() for k,l in results_loss.items() }
            else:
                assert False, 'Invalid type for loss'
            print_message = f"Epoch[{epoch}] Iteration[{iter_}/{num_train_batches}] Loss: "+"".join([ f" {k} = {v:.4f} " for k,v in loss_val.items() ])
            
        print(print_message)

ev = Process(func=log_training_loss)

em.register_event("iter-end", ev)

em._set_data_server()

In [ ]:

from ignite.engine import Events
from ignite.handlers import EarlyStopping, ModelCheckpoint

tensorboardX_flags = ['t', 'tf', 'tensorboard', 'tensorboardX']
visdom_flags = ['v', 'vis', 'visdom']

"""
    ToDo
        * correct tensorboard flow
        * correct visdom flow
        * add tensorwatch flow
    Investigate
    * flush setting on tdqm ?
"""


def create_default_events(config):
    vis_tool = config['others']['vis_tool']
    if vis_tool in tensorboardX_flags:
        try:
            from tensorboardX import SummaryWriter
        except ImportError:
            raise RuntimeError("No tensorboardX package is found. Please install with the command: \npip install tensorboardX")
        log_dir = "tf_log"
    elif vis_tool in visdom_flags:
        try:
            import visdom
        except ImportError:
            raise RuntimeError("No visdom package is found. Please install it with command: \n pip install visdom")
    else:
        if vis_tool == 'notebook':
            from tqdm import tqdm_notebook as tqdm
        else:
            from tqdm import tqdm


    objects = config['objects']
    grad_accumulation_steps = config['others'].get('grad_accumulation_steps', 1)
    train_loader = objects['data']['train_loader']
    val_loader = objects['data']['val_loader']
    eval_train_loader = objects['data'].get('eval_train_loader', None)
    train_evaluator = objects['engine']['train_evaluator']
    val_evaluator = objects['engine']['val_evaluator']
    trainer = objects['engine']['trainer']

    num_train_batches = len(train_loader)
    log_interval = config['others']['log_interval']
    
    if vis_tool in tensorboardX_flags:
        def create_summary_writer(log_dir):
            writer = SummaryWriter(logdir=log_dir)
            #data_loader_iter = iter(data_loader)
            #x, y = next(data_loader_iter)
            #try:
            #    writer.add_graph(model, x)
            #except Exception as e:
            #    print("Failed to save model graph: {}".format(e))
        return writer
        writer = create_summary_writer(log_dir)
        objects['vis_tool'] = writer
    elif vis_tool in visdom_flags:
        vis = visdom.Visdom()
        def create_plot_window(vis, xlabel, ylabel, title):
            return vis.line(X=np.array([1]), Y=np.array([np.nan]), opts=dict(xlabel=xlabel, ylabel=ylabel, title=title))
        train_loss_window = create_plot_window(vis, '#Iterations', 'Loss', 'Training Loss')
        train_avg_loss_window = create_plot_window(vis, '#Iterations', 'Loss', 'Training Average Loss')
        train_avg_accuracy_window = create_plot_window(vis, '#Iterations', 'Accuracy', 'Training Average Accuracy')
        val_avg_loss_window = create_plot_window(vis, '#Epochs', 'Loss', 'Validation Average Loss')
        objects['vis_tool'] = vis
    else:
        desc = "ITERATION - loss: "
        pbar = tqdm(
            initial=0, leave=False, total=len(train_loader),
        )
        objects['vis_tool'] = pbar

    import torch
    def log_training_loss(self, state, event_config={}):
        """
            ### ToDo : check the following process
        """
        epcoh = state["global/epoch"]
        iter_ = state["run/iteration"]
        iter_ = ((iter_ - 1) % num_train_batches) // grad_accumulation_steps + 1
        #num_iter_per_epoch = (num_train_batches - 1) // grad_accumulation_steps + 1

        if iter_ % log_interval == 0:
            results = self.results
            results_loss = results['loss']
            if isinstance(results_loss, torch.Tensor):
                loss_val = results_loss.item()
                print_message = "Epoch[{}] Iteration[{}/{}] Loss: {:.2f}".format(epoch, iter_, num_train_batches, loss_val)
            else:
                if isinstance(results_loss, list):
                    loss_val = { f"({n})":l.item() for n,l in enumerate(results_loss, 1) }
                elif isinstance(results_loss, dict):
                    loss_val = { k:l.item() for k,l in results_loss.items() }
                else:
                    assert False, 'Invalid type for loss'
                print_message = f"Epoch[{epoch}] Iteration[{iter_}/{num_train_batches}] Loss: "+"".join([ f" {k} = {v:.4f} " for k,v in loss_val.items() ])

            if vis_tool in tensorboardX_flags:
                print(print_message, flush=True)
                writer.add_scalar("training/loss", loss_val, iter_)
            elif vis_tool in visdom_flags:
                """
                    to correct in th future
                """
                print(print_message, flush=True)
                vis.line(X=np.array([state.iteration]),
                         Y=np.array([state.output]), update='append', win=train_loss_window)
            else:
                pbar.desc = print_message
                pbar.update(log_interval)

    def log_training_results(state, **kwargs):
        if vis_tool not in tensorboardX_flags and visdom_flags not in visdom_flags:
            pbar.refresh()
        train_evaluator.run(train_loader if eval_train_loader is None else eval_train_loader)
        metrics = train_evaluator.state.metrics
        print_logs(config, state, metrics, phase='train')

    def log_validation_results(state, **kwargs):
        val_evaluator.run(val_loader)
        metrics = val_evaluator.state.metrics
        print_logs(config, state, metrics, phase='validation')
        if vis_tool not in tensorboardX_flags and visdom_flags not in visdom_flags:
            pbar.n = pbar.last_print_n = 0
        ###
        trainer.save_metrics()

    

    if "handlers" in config.keys():
        handlers =  config['handlers']

        if 'early_stopping' in handlers.keys():
            hdl_early_stopping = handlers['early_stopping']
            patience = hdl_early_stopping['patience']
            if 'score_function' in hdl_early_stopping.keys():
                score_function = hdl_early_stopping['score_function']
            else:
                score_name = hdl_early_stopping.get('score_name', 'loss')
                score_function = lambda engine:-engine.state.metrics[score_name]
            ES_handler = EarlyStopping(patience=patience, score_function=score_function, trainer=trainer)
            val_evaluator.add_event_handler(Events.COMPLETED, ES_handler)

        if 'checkpoint' in handlers.keys():
            hdl_checkpoint = handlers['checkpoint']
            save_interval = hdl_checkpoint.get('save_interval',1)
            n_saved = hdl_checkpoint.get('n_saved', float('inf'))
            target_models = hdl_checkpoint.get('target_models', list(objects['models'].keys()))
            save_target_models = { model_name : objects['models'][model_name] for model_name in target_models }
            model_name_prefix = hdl_checkpoint.get('prefix', "")
            model_name = hdl_checkpoint.get('name', "model")
            model_dir = hdl_checkpoint.get('save_dir', "checkpoints")

            #model = config['objects']['model']
            MC_handler = ModelCheckpoint(model_dir, model_name_prefix, save_interval=save_interval, n_saved=n_saved, create_dir=True)
            trainer.add_event_handler(Events.EPOCH_COMPLETED, MC_handler, save_target_models)#{model_name : model})


def print_logs(config, state, metrics, phase):
    vis_tool = config['others']['vis_tool']
    objects = config['objects']
    metrics_log = objects.get('metrics_log', None)

    if metrics_log is not None:
        for name, value in metrics.items():
            metrics_log[f'{phase}/{name}'].append(value)

    epoch = state.epoch
    
    print_message = f"Epoch: {epoch} :: {phase:^12} results - " +\
        " ".join([f"{name}: {value:.4f}" for name, value in metrics.items() if isinstance(value, float)])
    
    if vis_tool in tensorboardX_flags:
        print(print_message, flush=True)
        for name, val in [("avg_loss", avg_nll), ("avg_accuracy", avg_accuracy)]:
            writer.add_scalar(f"{phase}/{name}", val, epoch)
    elif vis_tool in visdom_flags:
        print(print_message, flush=True)
        for val, win in [(avg_accuracy, val_avg_accuracy_window), (avg_nll, val_avg_loss_window)]:
            vis.line(X=np.array([epoch]), Y=np.array([val]), win=win, update='append')
    else:
        from tqdm import tqdm
        tqdm.write(print_message)

    
def close_logger(vis_tool, vis_tool_name):
    if vis_tool_name in tensorboardX_flags:
        writer = vis_tool
        writer.close()
    elif vis_tool_name in visdom_flags:
        vis = vis_tool
        pass
    else:
        pbar = vis_tool
        pbar.close()